In [45]:
import pandas as pd
import os

# Longitud de la secuencia precedente para recolectar para la RNN
SEQ_LEN = 60

# Cuánto tiempo en el futuro estamos tratando de predecir
FUTURE_PERIOD_PREDICT = 3

# El ratio que queremos predecir
RATIO_TO_PREDICT = "DOGE"

# Función para clasificar si el valor futuro es mayor que el valor actual
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

# Uso de una cadena sin procesar para evitar errores de escape de unicode
file_path = r"C:\Users\CRISTIAN CHAVEZ\Proyecto IA\cripto\criptodata\DOGE.xlsx"

# Leer el archivo inicial
df = pd.read_excel(file_path, names=['time', 'open', 'high', 'low', 'close', 'volume', 'usdVolume'])

# DataFrame principal vacío para almacenar todos los datos
main_df = pd.DataFrame()

# Las 4 criptomonedas que queremos considerar
ratios = ["BTC", "DOGE", "ETH", "QTU"]

# Ruta base donde se encuentran los archivos de datos
base_path = r"C:\Users\CRISTIAN CHAVEZ\Proyecto IA\cripto\criptodata"

# Iterar sobre cada ratio
for ratio in ratios:
    print(ratio)
    
    # Obtener la ruta completa al archivo
    dataset = os.path.join(base_path, f"{ratio}.xlsx")
    
    # Leer el archivo específico
    df = pd.read_excel(dataset, names=['time', 'open', 'high', 'low', 'close', 'volume', 'usdVolume'])
    
    # Renombrar columnas de 'close' y 'volume' para incluir el ticker
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    # Establecer 'time' como índice para poder unirlos por este tiempo compartido
    df.set_index("time", inplace=True)
    
    # Ignorar otras columnas además de 'close' y 'volume'
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    # Si el DataFrame principal está vacío, simplemente asignar el DataFrame actual
    if len(main_df) == 0:
        main_df = df
    else:
        # De lo contrario, unir estos datos con el DataFrame principal
        main_df = main_df.join(df)

# Crear una nueva columna 'future' que es el valor de cierre futuro desplazado por 'FUTURE_PERIOD_PREDICT'
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

# Crear una columna 'target' que clasifica si el valor futuro es mayor que el valor actual
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

# Imprimir las primeras filas del DataFrame resultante
print(main_df.head())


BTC
DOGE
ETH
QTU
            BTC_close  BTC_volume  DOGE_close  DOGE_volume  ETH_close  \
time                                                                    
1715356800    60793.4      15.101    0.144841     169877.0    2914.04   
1715356860    60729.3       6.395    0.144513     245666.0    2907.25   
1715356920    60744.5       7.272    0.144437     417310.0    2906.65   
1715356980    60713.9       4.568    0.144207     311595.0    2904.65   
1715357040    60683.8      18.586    0.144067     153653.0    2901.04   

            ETH_volume  QTU_close  QTU_volume    future  target  
time                                                             
1715356800        45.7      3.629      1283.8  0.144207       0  
1715356860        23.8      3.620        44.7  0.144067       0  
1715356920       997.9      3.611        20.8  0.143458       0  
1715356980        82.0      3.606        46.6  0.143589       0  
1715357040        96.0      3.609        32.8  0.143913       0  
